In [1]:
%matplotlib inline 
import matplotlib.pylab
import pandas as pd
import numpy as np

In [2]:
with open('data/ao_monthly.txt') as f:
    for x in range(5):
        print(next(f))

 1950    1  -0.60310E-01

 1950    2   0.62681E+00

 1950    3  -0.81275E-02

 1950    4   0.55510E+00

 1950    5   0.71577E-01



In [3]:
# Read a table of fixed-width formatted lines into DataFrame
data = pd.read_fwf('data/ao_monthly.txt', header = None)

In [4]:
# Not so great
data.head()

,0,1,2
0,1950,1,-0.060310
1,1950,2,0.626810
2,1950,3,-0.008128
3,1950,4,0.555100
4,1950,5,0.071577


### Look at the options for read_fwf...what looks relevant?
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_fwf.html

parse_dates : boolean or list of ints or names or list of lists or dict, default False
boolean. If True -> try parsing the index.
list of ints or names. e.g. If [1, 2, 3] -> try parsing columns 1, 2, 3 each as a separate date column.
list of lists. e.g. If [[1, 3]] -> combine columns 1 and 3 and parse as a single date column.
dict, e.g. {‘foo’ : [1, 3]} -> parse columns 1, 3 as date and call result ‘foo’.

In [5]:
# %load snippets/readtime.py
data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], infer_datetime_format = True)

In [6]:
data.head()

,2
0_1,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


In [7]:
data.index

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01',
               ...
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01'],
              dtype='datetime64[ns]', name='0_1', length=798, freq=None)

In [8]:
data.index.names = ['Month']
data.columns = ['Value']
data.head()

,Value
Month,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


### What is the empirical range of dates?

In [9]:
# %load snippets/daterange.py
min(data.index)

Timestamp('1950-01-01 00:00:00')

In [10]:
max(data.index)

Timestamp('2016-06-01 00:00:00')

In [11]:
type(data.index)

pandas.core.indexes.datetimes.DatetimeIndex

### How can we convert to complementary representation?

In [12]:
# %load snippets/changerep.py
data = data.to_period()
data

,Value
Month,
1950-01,-0.060310
1950-02,0.626810
1950-03,-0.008128
1950-04,0.555100
1950-05,0.071577
1950-06,0.538570
1950-07,-0.802480
1950-08,-0.851010
1950-09,0.357970


### More about reading in data

When reading in dates with a pd.read function, you have several time-related parameters you can adjust: 
parse_dates, infer_datetime_format, date_parser

Experiment with these using %timeit to see if there are performance differences

Hint:
infer_datetime_format = True, no date parser provided
What other combos can you come up with?

In [13]:
import timeit
# First, let's see how to use a date_parser:
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')

%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = dateparse)

100 loops, best of 3: 15.8 ms per loop


In [14]:
%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], infer_datetime_format = True)

100 loops, best of 3: 4.89 ms per loop


In [15]:
%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = dateparse, infer_datetime_format = True)

100 loops, best of 3: 15.8 ms per loop


### You can also extract datetimes from existing Data Frame columns

In [16]:
# new in pandas 0.18
df = pd.DataFrame({'year':[2015, 2016], 'month':[2,3], 'day':[4,5], 'hour':[12, 13]})
df

,day,hour,month,year
0,4,12,2,2015
1,5,13,3,2016


In [17]:
pd.to_datetime(df)

0   2015-02-04 12:00:00
1   2016-03-05 13:00:00
dtype: datetime64[ns]

### Truncating

In [18]:
ts = pd.Series(range(10), index = pd.date_range('7/31/15', freq = 'M', periods = 10))
ts

2015-07-31    0
2015-08-31    1
2015-09-30    2
2015-10-31    3
2015-11-30    4
2015-12-31    5
2016-01-31    6
2016-02-29    7
2016-03-31    8
2016-04-30    9
Freq: M, dtype: int32

In [19]:
# truncating preserves frequency
ts.truncate(before = '10/31/2015', after = '12/31/2015')

2015-10-31    3
2015-11-30    4
2015-12-31    5
Freq: M, dtype: int32

In [20]:
# You can truncate in a way that does not preserve frequency
ts[[1, 6, 7]].index

DatetimeIndex(['2015-08-31', '2016-01-31', '2016-02-29'], dtype='datetime64[ns]', freq=None)

In [21]:
# But Pandas will try to preserve frequency automatically whenever possible
ts[0:10:2]

2015-07-31    0
2015-09-30    2
2015-11-30    4
2016-01-31    6
2016-03-31    8
Freq: 2M, dtype: int32